In [ ]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import string

from sklearn.base import BaseEstimator
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from typing import Union
from xgboost import XGBRFClassifier

In [ ]:
# Loading CSVs into Pandas DataFrame
train_df = pd.read_csv('/kaggle/input/titanic/train.csv')
X_test = pd.read_csv('/kaggle/input/titanic/test.csv')

# Preprocessing

In [ ]:
def get_is_male(row: pd.Series) -> bool:
    """
    Creates boolen indicator value for men in Titanic data.

    Args:
        row (pd.Series): A row of Titanic data containing a 'Sex' column.

    Returns:
        Bool: True if Sex=male, False otherwise.
    """
    return row['Sex'] == 'male'

In [ ]:
def get_is_child(row: pd.Series) -> bool:
    """
    Creates boolen indicator value for childrean in Titanic data.  Defines a child
    to be <18.

    Args:
        row (pd.Series): A row of Titanic data containing a 'Age' column.

    Returns:
        Bool: True if Age<18, False otherwise.
    """
    return row['Age'] < 18

In [ ]:
def get_is_male_is_child(row: pd.Series) -> str:
    """
    Creates compositie indicator variable for validation set creation.  For this
    problem, we have the features for the test set.  The proportions of women and
    children in the training set is different than that in the test set.  This
    creates lower model preformance on the test features than the validation
    features.  We use this indicator to create a custom validation set based on the
    proportions of women and children in the test set.
    
    Args:
        row (pd.Series): A row of Titanic data containing 'is_male', and 'is_child'
        columns.
        
    Returns:
        str: Concatenated is_male and is_child columns from row.
    """
    return f"{row['is_male']}_{row['is_child']}"

## Creating Training and Validation Sets

In [ ]:
train_df['is_male'] = train_df.apply(get_is_male, axis=1)
X_test['is_male'] = X_test.apply(get_is_male, axis=1)

train_df['is_child'] = train_df.apply(get_is_child, axis=1)
X_test['is_child'] = X_test.apply(get_is_child, axis=1)

train_df['is_male_is_child'] = train_df.apply(get_is_male_is_child, axis=1)
X_test['is_male_is_child'] = X_test.apply(get_is_male_is_child, axis=1)

In [ ]:
# Creating X_val based on the proportion of women and children in X_test
val_df = pd.DataFrame()

test_proxy_proportions = X_test["is_male_is_child"].value_counts(normalize=True)
for proxy_group, proportion in test_proxy_proportions.items():
    group_data = train_df[train_df['is_male_is_child'] == proxy_group]
    n_samples = int(proportion * len(train_df) * 0.15)  # 15% for validation set
    # Sample without replacement from the group
    sampled_group_data = group_data.sample(n=n_samples, replace=False)
    val_df = pd.concat([val_df, sampled_group_data])

# Drop validation samples from training data to avoid data leakage.
train_df = train_df.drop(val_df.index)

In [ ]:
# Separating features and targets in training and validation sets.
y_train = train_df.Survived
X_train = train_df.drop(columns=['Survived'])

y_val = val_df.Survived
X_val = val_df.drop(columns=['Survived'])

## Creating Features

In [ ]:
def get_deck(row: pd.Series) -> str:
    """
    Gets the Deck of a Titanic Ticket.  The Deck of a Titanic Ticket is the first
    Character of the Ticket if there's a space in it.  Else 'Numeric'.

    Args:
        row (pd.Series): Row of titanic training data.

    Returns:
        str: Deck of a Titanic Ticket.
    """
    ticket = row['Ticket']
    if len(ticket.split()) > 1:
        return ticket[0]
    else:
        return 'Numeric'

In [ ]:
"""
TODO: train XGBoost with below parameters:

model_input_columns = ['Fare',
                       'family_bucket_1',
                       'family_bucket_2',
                       'family_bucket_3',
                       'has_cabin',
                       'is_male',
                       'has_age',
                       'age_bin',
                       'is_child',
                       'class_1',
                       'class_2',
                       'class_3'
                      ]
"""
def preprocess(X: pd.DataFrame) -> pd.DataFrame:
    """
    Generates relevant features to predict whether a passenger survived the
    Titanic disaster.
    
    Args:
        X (pd.DataFrame): Features.

    Returns:
        pd.DataFrame: Subset of X with engineered columns defined by features
            where multiclass variables are binarized.
    """
    # Making a deep copy of X to not alter our original training data.
    X_preprocessed = X.copy()

    X_preprocessed['Deck'] = X_preprocessed.apply(get_deck, axis=1)

    # Assuming all passengers with null ages are 40.
    X_preprocessed['Age_not_null'] = X_preprocessed['Age'].fillna(40)
    # Assuming all passengers will null Fares paid $0.
    X_preprocessed['Fare_not_null'] = X_preprocessed['Fare'].fillna(0)

    # Creating features based on Cabin.
    X_preprocessed['no_cabin'] = X_preprocessed.apply(
        lambda row: pd.isnull(row['Cabin']), axis=1)
    X_preprocessed['cabin_first_char'] = X_preprocessed['Cabin'].apply(
        lambda row: row[0] if pd.notnull(row) else row)

    # The size of a passenger's family is the number of siblings plus the number of
    # parents plus themselves.
    X_preprocessed['num_family_members'] = X_preprocessed['SibSp']+X_preprocessed['Parch']+1
    X_preprocessed['family_size'] = pd.cut(
        x=X_preprocessed['num_family_members'],
        bins=[0, 2, 5, float('inf')],
        labels=['alone', 'small', 'big'],
        right=False
    )

    # One hot encoding gender, embarkment location, Deck, cabin_first_char, and
    # family_size.
    X_preprocessed = pd.get_dummies(X_preprocessed, columns=[
        'Embarked', 'Deck', 'cabin_first_char', 'family_size'])
    return X_preprocessed

# Models

In [ ]:
def evaluate_model(model: BaseEstimator,
                   X_train: pd.DataFrame,
                   y_train: pd.Series,
                   X_val: pd.DataFrame,
                   y_val: pd.Series) -> dict[str: Union[str, float]]:
    """
    Returns accuracy on training and validation sets. 
    
    Args:
        X_train (pd.DataFrame): Training features.
        X_val (pd.DataFrame): Validation features.
    
    Returns:
        (dict[str: Union[str, float]]): dictionary of training and validation
            accuracies for model.
    """
    y_pred_train = model.predict(X_train)
    y_pred_val = model.predict(X_val)
    
    train_accuracy = accuracy_score(
        y_true=y_train, y_pred=y_pred_train)
    val_accuracy = accuracy_score(
        y_true=y_val, y_pred=y_pred_val)

    return {
        'model': model.estimator,
        'training_accuracy': train_accuracy,
        'validation_accuracy': val_accuracy
    }

## Gender Model

In [ ]:
class GenderModel(BaseEstimator):
    estimator = "GenderModel"
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series) -> "GenderModel":
        """
        Fits a GenderModel for the given training data.

        Args:
            X (pd.DataFrame): Pandas Dataframe containing 'Sex' column.
            y (pd.Series): Boolean target variable.

        Retruns:
            GenderModel: A fit GenderModel to the training data.
        """
        return self
    def predict(self, X: pd.DataFrame) -> np.ndarray:
        """
        Predicts whether a passenger survives the Titanic solely based on gender.
        Women are predicted to survive, men are not.
    
        Args:
            X (pd.DataFrame): Pandas Dataframe containing 'Sex' column.
    
        Returns:
            pd.Series: True if female, False otherwise.
        """
        return np.where(X['Sex'] == 'female', 1, 0)
    def score(self, X: pd.DataFrame, y_true: pd.DataFrame) -> float:
        """
        Returns the accuracy of predicting a GenderModel on X against y.

        Args:
            X (pd.DataFrame): Pandas Dataframe containing 'Sex' column.

        Returns:
            float: accuracy of predictinging a GenderModel on X against y.
        """
        y_pred = self.predict(X)
        return (y_pred == y_true).mean()

In [ ]:
gender_model = GridSearchCV(GenderModel(), param_grid={})

gender_model.fit(X_train, y_train)

## Random Forest
### Feature Selection

In [ ]:
features_rf = ["is_male", "Age_not_null", "Fare_not_null", "Pclass"]

X_train_preprocessed_rf = preprocess(X_train).reindex(
    columns=features_rf, fill_value=0)

# Ensuring columns in X_val_preprocessed_rf are the same as the columns in
# X_train_preprocessed_rf since we have one hot encodings
X_val_preprocessed_rf = preprocess(X_val).reindex(
    columns=features_rf, fill_value=0)

### Hyperparameter Tuning

In [ ]:
# Trying ~30 combinations takes about 45 seconds.
param_grid_rf = [{
    'max_depth': np.arange(start=1, stop=4, step=1),
    'max_features': np.arange(start=1, stop=4, step=1),
    'n_estimators': [150, 200, 250]
}]

start_time = datetime.datetime.now()
random_forest_model = GridSearchCV(
    estimator=RandomForestClassifier(),
    param_grid=param_grid_rf,
    cv=StratifiedKFold(n_splits=5, shuffle=True),
    # Using roc_auc as we have a class imbalance.
    scoring='roc_auc'
)
random_forest_model.fit(X_train_preprocessed_rf, y_train)
end_time = datetime.datetime.now()

gridsearch_time = end_time-start_time
print(f"Random Forest:\tGridSearchCV took {gridsearch_time}")

In [ ]:
evaluate_model(
    model=random_forest_model, X_train=X_train_preprocessed_rf,
    y_train=y_train, X_val=X_val_preprocessed_rf, y_val=y_val)

## XGBoost
### Feature Selection

In [ ]:
features_xgb = ["is_male", "Age", "Fare", "Pclass", "family_size_alone",
                "family_size_small", "family_size_big"]

X_train_preprocessed_xgb = preprocess(X_train).reindex(
    columns=features_xgb, fill_value=0)

# Ensuring columns in X_val_preprocessed_rf are the same as the columns in
# X_train_preprocessed_rf since we have one hot encodings
X_val_preprocessed_xgb = preprocess(X_val).reindex(
    columns=features_xgb, fill_value=0)

In [ ]:
# TODO: XGBoost is overfitting my training data.

# Trying ~100 combinations takes about 1 minute.
param_grid_xgb = [{
    'subsample': np.arange(0.3, 0.8, 0.1),
    'colsample_bynode': np.arange(0.3, 0.8, 0.1),
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'gamma': [0, 0.1, 0.2]
}]

start_time = datetime.datetime.now()
xgboost_model = GridSearchCV(
    estimator=XGBRFClassifier(),
    param_grid=param_grid_xgb,
    cv=StratifiedKFold(n_splits=5, shuffle=True),
    # Using roc_auc as we have a class imbalance.
    scoring='roc_auc'
)
xgboost_model.fit(X_train_preprocessed_xgb, y_train)
end_time = datetime.datetime.now()

gridsearch_time = end_time-start_time
print(f"XGBoost:\tGridSearchCV took {gridsearch_time}")

In [ ]:
xgboost_model.best_params_

In [ ]:
evaluate_model(model=xgboost_model, X_train=X_train_preprocessed_xgb,
               y_train=y_train, X_val=X_val_preprocessed_xgb, y_val=y_val)

### Leaderboard

In [ ]:
# Adding models to the Leaderboard.
gender_model_evaluation = evaluate_model(
    model=gender_model, X_train=X_train, y_train=y_train,
    X_val=X_val, y_val=y_val)

random_forest_evaluation = evaluate_model(
    model=random_forest_model, X_train=X_train_preprocessed_rf, y_train=y_train,
    X_val=X_val_preprocessed_rf, y_val=y_val)

xgb_evaluation = evaluate_model(
    model=xgboost_model, X_train=X_train_preprocessed_xgb,
    y_train=y_train, X_val=X_val_preprocessed_xgb, y_val=y_val)

In [ ]:
leaderboard = pd.DataFrame([
gender_model_evaluation, random_forest_evaluation, xgb_evaluation])

leaderboard.sort_values(by='validation_accuracy', ascending=False, inplace=True)

leaderboard.head(len(leaderboard))

In [ ]:
# Predicting on X_test.
X_test_preprocessed_rf = preprocess(X_test).reindex(
    columns=features_rf, fill_value=0)
y_pred_test = random_forest_model.predict(X_test_preprocessed_rf)

submission_df = pd.DataFrame({
    "PassengerId": X_test.PassengerId,
    "Survived": y_pred_test
})

submission_df.head()

In [ ]:
submission_df.to_csv('submission.csv', index=False)
print("Submission saved to submission.csv")